# Introduction to Cross-Validation - Lab

## Introduction

In this lab, you'll be able to practice your cross-validation skills!


## Objectives

You will be able to:

- Compare the results with normal holdout validation
- Apply 5-fold cross validation for regression

## Let's get started

This time, let's only include the variables that were previously selected using recursive feature elimination. We included the code to preprocess below.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.datasets import load_boston

In [3]:
boston = load_boston()
boston #we can think of this as capital x, target can be used as lower case y 

{'data': array([[6.3200e-03, 1.8000e+01, 2.3100e+00, ..., 1.5300e+01, 3.9690e+02,
         4.9800e+00],
        [2.7310e-02, 0.0000e+00, 7.0700e+00, ..., 1.7800e+01, 3.9690e+02,
         9.1400e+00],
        [2.7290e-02, 0.0000e+00, 7.0700e+00, ..., 1.7800e+01, 3.9283e+02,
         4.0300e+00],
        ...,
        [6.0760e-02, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9690e+02,
         5.6400e+00],
        [1.0959e-01, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9345e+02,
         6.4800e+00],
        [4.7410e-02, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9690e+02,
         7.8800e+00]]),
 'target': array([24. , 21.6, 34.7, 33.4, 36.2, 28.7, 22.9, 27.1, 16.5, 18.9, 15. ,
        18.9, 21.7, 20.4, 18.2, 19.9, 23.1, 17.5, 20.2, 18.2, 13.6, 19.6,
        15.2, 14.5, 15.6, 13.9, 16.6, 14.8, 18.4, 21. , 12.7, 14.5, 13.2,
        13.1, 13.5, 18.9, 20. , 21. , 24.7, 30.8, 34.9, 26.6, 25.3, 24.7,
        21.2, 19.3, 20. , 16.6, 14.4, 19.4, 19.7, 20.5, 25. , 23.4, 18.9,
        35.4, 24.7, 3

In [5]:
boston_features = pd.DataFrame(boston.data, 
                               columns=boston.feature_names)

In [7]:
boston_features.head().T

,0,1,2,3,4
CRIM,0.00632,0.02731,0.02729,0.03237,0.06905
ZN,18.00000,0.00000,0.00000,0.00000,0.00000
INDUS,2.31000,7.07000,7.07000,2.18000,2.18000
CHAS,0.00000,0.00000,0.00000,0.00000,0.00000
NOX,0.53800,0.46900,0.46900,0.45800,0.45800
RM,6.57500,6.42100,7.18500,6.99800,7.14700
AGE,65.20000,78.90000,61.10000,45.80000,54.20000
DIS,4.09000,4.96710,4.96710,6.06220,6.06220
RAD,1.00000,2.00000,2.00000,3.00000,3.00000
TAX,296.00000,242.00000,242.00000,222.00000,222.00000


In [ ]:
b = boston_features["B"]
logdis = np.log(boston_features["DIS"])
loglstat = np.log(boston_features["LSTAT"])

# minmax scaling
boston_features["B"] = (b-min(b))/(max(b)-min(b))
boston_features["DIS"] = (logdis-min(logdis))/(max(logdis)-min(logdis))

#standardization
boston_features["LSTAT"] = (loglstat-np.mean(loglstat))/np.sqrt(np.var(loglstat))

In [11]:
X = boston_features.values 
y = boston.target

## Train test split

Perform a train-test-split with a test set of 0.20.

In [12]:
from sklearn.model_selection import train_test_split

In [15]:
train_test_split

<function sklearn.model_selection._split.train_test_split(*arrays, **options)>

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = .2)

Fit the model and apply the model to the make test set predictions

In [17]:
from sklearn.linear_model import LinearRegression #linearregression is my class

In [18]:
model = LinearRegression()

In [19]:
model.fit?

In [20]:
model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [29]:
model.coef_

array([-9.11406074e-02,  4.13450758e-02,  1.79556953e-02,  2.45106856e+00,
       -1.67317008e+01,  3.88062994e+00, -1.03300363e-03, -1.38962559e+00,
        2.88120142e-01, -1.35172045e-02, -9.16538066e-01,  8.02705494e-03,
       -4.93228118e-01])

In [28]:
list(zip(boston_features.columns, model.coef_))

[('CRIM', -0.09114060736334649),
 ('ZN', 0.04134507580700985),
 ('INDUS', 0.017955695279633286),
 ('CHAS', 2.451068563236064),
 ('NOX', -16.731700795575694),
 ('RM', 3.8806299429882882),
 ('AGE', -0.0010330036331479056),
 ('DIS', -1.3896255874390055),
 ('RAD', 0.288120142404743),
 ('TAX', -0.013517204545595157),
 ('PTRATIO', -0.9165380655688997),
 ('B', 0.0080270549445787),
 ('LSTAT', -0.4932281175929743)]

Calculate the residuals and the mean squared error

In [22]:
#y_pred is what i'm predicting y to be
y_pred  = model.predict(X_test)

In [23]:
residuals = y_test - y_pred

In [24]:
squared_error = residuals ** 2

In [27]:
mse = squared_error.mean()
mse

29.54432658237773

## Cross-Validation: let's build it from scratch!

### Create a cross-validation function

Write a function k-folds that splits a dataset into k evenly sized pieces.
If the full dataset is not divisible by k, make the first few folds one larger then later ones.

We want the folds to be a list of subsets of data!

In [ ]:
def kfolds(data, k):
    # Force data as pandas dataframe
    # add 1 to fold size to account for leftovers           
    return None

### Apply it to the Boston Housing Data

In [ ]:
# Make sure to concatenate the data again

### Perform a linear regression for each fold, and calculate the training and test error

Perform linear regression on each and calculate the training and test error.

In [ ]:
test_errs = []
train_errs = []
k=5

for n in range(k):
    # Split in train and test for the fold
    train = None
    test = None
    # Fit a linear regression model
    
    #Evaluate Train and Test Errors

# print(train_errs)
# print(test_errs)

## Cross-Validation using Scikit-Learn

This was a bit of work! Now, let's perform 5-fold cross-validation to get the mean squared error through scikit-learn. Let's have a look at the five individual MSEs and explain what's going on.

Next, calculate the mean of the MSE over the 5 cross-validations and compare and contrast with the result from the train-test-split case.

##  Summary 

Congratulations! You now practiced your knowledge on k-fold crossvalidation!